In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import altair as alt

In [3]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [127]:
uk = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/inflation/series-260825.csv', skiprows=501).rename(columns={'2014 DEC': 'Date',"0.5": "Value"})
eu = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/inflation/ECB Data Portal_20250826162819.csv', skiprows=216).rename(columns={'2014-12-31': 'Date',"2014Dec": "drop","-0.2":"Value"})
us = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/inflation/CPIAUCSL.csv').rename(columns={'observation_date': 'Date',"CPIAUCSL":"raw value"})    

In [128]:
uk['Date'] = pd.to_datetime(uk['Date'], format='%Y %b')

In [189]:
uk

,Date,Value
0,2015-01-01,0.3
1,2015-02-01,0.0
2,2015-03-01,0.0
3,2015-04-01,-0.1
4,2015-05-01,0.1
...,...,...
122,2025-03-01,2.6
123,2025-04-01,3.5
124,2025-05-01,3.4
125,2025-06-01,3.6


In [129]:
eu = eu[['Date','Value']]
eu['Date'] = pd.to_datetime(eu['Date']).dt.to_period('M').dt.start_time

In [ ]:
us['Date'] = pd.to_datetime(us['Date'], format='%Y-%m-%d')
us['Value'] = 0
for i in range(0,len(us)):
    us['Value'].iloc[i] = ((us['raw value'].iloc[i] - us['raw value'].iloc[i - 12]) / us['raw value'].iloc[i - 12]) * 100
us = us[['Date','Value']][12:]
us['Value'] = us['Value'].round(1)

In [ ]:
infl = uk.merge(eu, how='outer', on='Date').merge(us, how='outer', on='Date').rename(columns={'Value_x': 'UK', 'Value_y': 'EU', 'Value': 'US'})
infl = infl.melt('Date', var_name='Country', value_name='Value')
infl['Value'] = infl['Value'] / 100
infl = infl[infl['Date'] >= '2024-08-01']
infl.so

In [193]:
# Custom sort so drawing order is US, EU, UK (UK will be drawn last/on top)
country_order = {'US': 0, 'EU': 1, 'UK': 2}
infl['sort_order'] = infl['Country'].map(country_order)
infl = infl.sort_values(['sort_order', 'Date']).drop('sort_order', axis=1)

In [202]:
# 2. Find last year for each Type
last_points = infl.loc[infl.groupby('Country')['Date'].idxmax()]

line = alt.Chart(infl).mark_line().encode(
    x=alt.X('Date', axis=alt.Axis(format='%b%y', title=None, labelAngle=0, tickCount={'interval': 'month', 'step': 1})),
    y=alt.Y('Value', title='Headline Inflation', axis=alt.Axis(format=',.1%', titleX=-87, titleY=-10)),
    color='Country'
)

# 4. Add text labels at the end
text = alt.Chart(last_points).mark_text(
    align='left',
    dx=5,
    fontWeight='bold',
    fontSize=13
).encode(
    x=alt.X('Date'),
    y=alt.Y('Value'),
    text='Country',
    color=alt.Color('Country', legend=None)
)

footnote = alt.Chart(pd.DataFrame({'text': [
    "Sources: ONS, ECB, FRED"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=10,
    fontStyle='italic',
    dy=175,
    x=-15
).encode(
    text='text'
)

# 5. Combine and display
chart = (line + text + footnote).properties(title= 'Headline Inflation - UK, US, and EU', width=600).configure_title(
    fontSize=20  # Increase title font size here
)
chart

alt.LayerChart(...)